In [6]:
# General imports and widgets
import os
if os.getcwd().split('/')[-1] != 'deepARTransit':
    os.chdir('../..')
import sys
import warnings
import numpy as np
import matplotlib.pylab as plt
from pixlc import plotting
import ipywidgets as widgets

plt.rcParams['figure.figsize'] = (16, 6)
plt.rcParams['font.size'] = 15
warnings.simplefilter("ignore")

# Project dir selection widget

project_button = widgets.Dropdown(
    options=[os.path.join('.'), # local git repo
             os.path.join('/','Users','Mario','mount_test','deepARTransit') # cobweb         
],
    #value=experiment_folder,
    # rows=10,
    description='\t\t',
    disabled=False
)

# Experiment selection widget

experiment_button = widgets.Dropdown(
    options=[os.path.join('cobweb', 'deeparsys_fulltime'),
             os.path.join('cobweb', 'deepartrans_fulltime'), 
             os.path.join('cobweb', 'rlc', 'deeparsys_nocov'),
             os.path.join('cobweb', 'rlc', 'deeparsys_cent'),
             os.path.join('cobweb', 'rlc', 'deeparsys_cent_2'),
             os.path.join('cobweb', 'rlc','deeparsys_pix'),
             os.path.join('cobweb', 'rlc','deeparsys_cent_pix'),
             os.path.join('cobweb', 'rlc','deeparsys_cent_24537600'),
             os.path.join('cobweb', 'rlc','deeparsys_cent_22808832'),
             os.path.join('cobweb', 'rlc','deeparsys_cent_27773440'),
],
    #value=experiment_folder,
    # rows=10,
    description='\t\t',
    disabled=False
)

In [7]:
#project directory for loading the data
project_button

Dropdown(description='\t\t', options=('.', '/Users/Mario/mount_test/deepARTransit'), value='.')

In [8]:
#Select an experime
experiment_button

Dropdown(description='\t\t', options=('cobweb/deeparsys_fulltime', 'cobweb/deepartrans_fulltime', 'cobweb/rlc/…

In [73]:
from utils.config import get_config_file, process_config
from deepartransit.data_handling import data_generator

experiment_folder = experiment_button.value
project_dir = project_button.value
experiment_dir = os.path.join(project_dir, 'deepartransit', 'experiments', experiment_folder)
print(experiment_dir)
config_file = get_config_file(experiment_dir)
config = process_config(config_file)
print('Data and covariates shapes:')
data = data_generator.DataGenerator(config)
width = int(np.sqrt(data.Z.shape[0]))



/Users/Mario/mount_test/deepARTransit/deepartransit/experiments/cobweb/rlc/deeparsys_cent
Data and covariates shapes:
(1, 690, 1) (1, 690, 2)
time_path parameter not found in config. Default to 0,1,2....T-1


In [74]:
os.listdir(os.path.join(project_dir, config.output_dir))

['loc_array_10.npy',
 'scales_array_10.npy',
 'pred_array_10.npy',
 'trans_pars_10.npy',
 'loc_array_20.npy',
 'scales_array_20.npy',
 'pred_array_20.npy',
 'trans_pars_20.npy',
 'loc_array_30.npy',
 'scales_array_30.npy',
 'pred_array_30.npy',
 'trans_pars_30.npy',
 'loc_array_40.npy',
 'scales_array_40.npy',
 'pred_array_40.npy',
 'trans_pars_40.npy',
 'loc_array_50.npy',
 'scales_array_50.npy',
 'pred_array_50.npy',
 'trans_pars_50.npy',
 'pred_array.npy',
 'loc_array_60.npy',
 'scales_array_60.npy',
 'pred_array_60.npy',
 'trans_pars_60.npy',
 'loc_array_70.npy',
 'scales_array_70.npy',
 'pred_array_70.npy',
 'trans_pars_70.npy',
 'loc_array_80.npy',
 'scales_array_80.npy',
 'pred_array_80.npy',
 'trans_pars_80.npy',
 'loc_array_90.npy',
 'scales_array_90.npy',
 'pred_array_90.npy',
 'trans_pars_90.npy',
 'loc_array_100.npy',
 'scales_array_100.npy',
 'pred_array_100.npy',
 'trans_pars_100.npy']

In [75]:
d = {}
for fn in [s for s in os.listdir(os.path.join(project_dir, config.output_dir)) if (s[:4] == 'pred')]:
    step = fn.split('_')[-1].split('.')[0]
    try:
        d[int(step)] = np.load(os.path.join(project_dir, config.output_dir, fn))#.mean(axis=0)
        #pred_instr = traces_array.mean(axis=(0))
        #length_pred = pred_instr.shape[1]
        #plc_trans = data.Z[:, config.pretrans_length:config.pretrans_length+config.trans_length+1]
    except:
        print(fn)
        continue

pred_array.npy


In [76]:
step_list = sorted([int(k) for k in d if k!='array'])
#sorted_preds = [d[k] for k in step_list]
#sorted_preds[0].shape, len(sorted_preds)
print(step_list)

[10, 20, 30, 40, 50, 60, 70, 80, 90, 100]


In [77]:
#plt.plot(d[270][:,0,:,0].T)

In [96]:
# Visualize evolution of predictions and fits
from ipywidgets import interact
from utils.transit import transit_linear

pred_range = range(config.pretrans_length, config.pretrans_length+ config.trans_length + 1)

def update(k = step_list[-1]):#(50, 400, 50)):
    loc_k = np.load(os.path.join(project_dir, config.output_dir, 'loc_array_{}.npy'.format(k))).flatten()
    
    t_c, delta, T, tau = np.load(os.path.join(project_dir, config.output_dir, 'trans_pars_{}.npy'.format(k)))
    transit = transit_linear(data.time_array[pred_range], t_c, delta, T, tau)
    
    scales_k = np.load(os.path.join(project_dir, config.output_dir, 'scales_array_{}.npy'.format(k))).flatten()
    plt.plot(data.Z.flatten())
    plt.plot()
    p = plt.plot(loc_k)
    plt.fill_between(data.time_array, loc_k - 2 * scales_k, loc_k + 2* scales_k, alpha= 0.3, color=p[0].get_color())
    plt.plot(pred_range, d[k][:,0,:,0].T, label=k, color='grey', linewidth=0.2, alpha= 0.3)
    plt.plot(pred_range, d[k][:,0,:,0].mean(0), label=k, color='red', linewidth=1)
    plt.plot(pred_range, data.scaler_Z.transform(data.scaler_Z.inverse_transform(d[k][:,0,:,0].mean(0)) * transit).flatten(), 
             label=k, color='red', linewidth=1, alpha=0.7)
    
    print('transit depth: {}'.format(delta))
    plt.title('step {}'.format(k))
    plt.show()
    #plt.figure
    #plt.plot(transit)
    #plt.show()

interact(update, k=(step_list[0], step_list[-1], config.freq_eval))
pass

interactive(children=(IntSlider(value=100, description='k', min=10, step=10), Output()), _dom_classes=('widget…